In [1]:
from os import listdir
from os.path import join

import numpy as np
import pandas as pd

import behavioural_models as bm

In [49]:
models = [
    bm.models.ExpectedUtility,
    bm.models.ProspectTheory,
    bm.mdft.MDFT,
    bm.models.GazeBaselineStat,
    bm.models.GazeBaselineDyn,
    bm.models.Glickman1Layer,
]

# Create synthetic DataFrame

For each real participant, we create data from $n$ synthetic agents, where $n$ is the number of models included.
Each synthetic data set has the same size as that of a real participant (i.e., 225 trials).

In [4]:
dummy_data = pd.read_csv(
    join(
        "..",
        "results",
        "3-behavioural-modeling",
        "predictions",
        "eu",
        "predictions_eu_1_de1.csv",
    )
)

In [41]:
# Build a dataframe with synthetic data from model predictions
# This dataframe will be len(models) times as large as the original dataframe,
# because for each real participant, len(models) new participants are generated
trials_synth = []

for m, model in enumerate(models):

    model_instance = model(data=dummy_data)
    model_label = model_instance.label.lower()

    files = [
        file
        for file in listdir(
            join("..", "results", "3-behavioural-modeling", "estimates", model_label)
        )
        if file.endswith(".csv")
    ]

    for file in files:

        _, _, subject, label = file.split("_")
        label = label.split(".")[0]
        new_subject = "-".join([subject, model_label])

        # Read model-predictions and select only the first simulation (`rep == 0`)
        pred_s = pd.read_csv(
            join(
                "..",
                "results",
                "3-behavioural-modeling",
                "predictions",
                model_label,
                f"predictions_{model_label}_{subject}_{label}.csv",
            ),
        )
        pred_s = pred_s.loc[pred_s["rep"] == 0]
        if len(pred_s) == 0:
            raise ValueError(f"Missing predictions for subject {new_subject}!")
        pred_s["subject"] = new_subject

        # Read estimates (= generating parameters)
        est_s = pd.read_csv(
            join(
                "..",
                "results",
                "3-behavioural-modeling",
                "estimates",
                model_label,
                f"estimates_{model_label}_{subject}_{label}.csv",
            ),
            index_col=0,
        )
        for parameter in model_instance.parameter_names:
            pred_s[parameter + "_gen"] = est_s[parameter].values[0]

        ## Add estimates to prediction DataFrame
        # Add to synthetic DataFrame
        trials_synth.append(pred_s)

trials_synth = (
    pd.concat(trials_synth)
    .reset_index(drop=True)
    .drop(
        [
            "key",
            "choice",
            "choice_tcd",
            "rt",
            "model",
            "rep",
            "pos_A",
            "pos_B",
            "pos_C",
            "pA_top",
            "pB_top",
            "pC_top",
        ],
        axis=1,
    )
    .rename({"predicted_choice": "choice"}, axis=1)
)

eu		40
pt		40
mdft		40
gaze-baseline-stat		40
gaze-baseline-dyn		40
glickman1layer		40


In [42]:
len(trials_synth["subject"].unique())

240

In [43]:
trials_synth.tail()

,subject,block,trial,effect,target,pA,pB,pC,mA,mB,...,alpha_gen,beta_gen,gamma_gen,w_gen,wd_gen,phi1_gen,phi2_gen,sig2_gen,lam_gen,theta_gen
53929,42-glickman1layer,3,221,attraction,A,0.72,0.65,0.70,10,14,...,0.486475,4.145332,0.456715,NaN,NaN,NaN,NaN,NaN,0.296674,0.69392
53930,42-glickman1layer,3,222,attraction,B,0.75,0.65,0.63,10,14,...,0.486475,4.145332,0.456715,NaN,NaN,NaN,NaN,NaN,0.296674,0.69392
53931,42-glickman1layer,3,223,attraction,B,0.78,0.65,0.63,11,15,...,0.486475,4.145332,0.456715,NaN,NaN,NaN,NaN,NaN,0.296674,0.69392
53932,42-glickman1layer,3,224,compromise,A,0.78,0.62,0.85,10,15,...,0.486475,4.145332,0.456715,NaN,NaN,NaN,NaN,NaN,0.296674,0.69392
53933,42-glickman1layer,3,225,attraction,A,0.78,0.68,0.76,10,15,...,0.486475,4.145332,0.456715,NaN,NaN,NaN,NaN,NaN,0.296674,0.69392


In [46]:
!mkdir ../results/S_recoveries

mkdir: ../results/S_recoveries: File exists


In [47]:
trials_synth.to_csv(join("..", "results", "S_recoveries", "trials_synth.csv"))

In [5]:
trials_synth = pd.read_csv(
    join("..", "results", "S_recoveries", "trials_synth.csv"), index_col=0
)

In [6]:
trials_synth.head()

,subject,block,trial,effect,target,pA,pB,pC,mA,mB,...,alpha_gen,beta_gen,gamma_gen,w_gen,wd_gen,phi1_gen,phi2_gen,sig2_gen,lam_gen,theta_gen
0,26-eu,1,1,attraction,A,0.72,0.65,0.70,11,15,...,0.329305,6.560476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,26-eu,1,2,filler,NaN,0.16,0.46,0.87,62,21,...,0.329305,6.560476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,26-eu,1,3,attraction,B,0.75,0.68,0.66,11,14,...,0.329305,6.560476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,26-eu,1,4,compromise,B,0.72,0.62,0.58,11,14,...,0.329305,6.560476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,26-eu,1,5,attraction,B,0.72,0.62,0.60,12,15,...,0.329305,6.560476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
trials_synth.columns

Index(['subject', 'block', 'trial', 'effect', 'target', 'pA', 'pB', 'pC', 'mA',
       'mB', 'mC', 'dwell_A', 'dwell_B', 'dwell_C', 'dwell_p', 'dwell_m',
       'dwell_Ap', 'dwell_Am', 'dwell_Bp', 'dwell_Bm', 'dwell_Cp', 'dwell_Cm',
       'dwell_total', 'dwell_target', 'dwell_competitor', 'dwell_decoy',
       'fixated_alternatives', 'fixated_attributes', 'fixation_durations',
       'choice', 'predicted_choiceprob_A', 'predicted_choiceprob_B',
       'predicted_choiceprob_C', 'alpha_gen', 'beta_gen', 'gamma_gen', 'w_gen',
       'wd_gen', 'phi1_gen', 'phi2_gen', 'sig2_gen', 'lam_gen', 'theta_gen'],
      dtype='object')

# Fix eye tracking columns which have arrays in them

In [70]:
trials_synth["fixated_alternatives"] = trials_synth["fixated_alternatives"].apply(
    lambda x: np.fromstring(x[1:-1], sep=" ", dtype=int)
)

trials_synth["fixated_attributes"] = trials_synth["fixated_attributes"].apply(
    lambda x: np.fromstring(x[1:-1], sep=" ", dtype=int)
)

trials_synth["fixation_durations"] = trials_synth["fixation_durations"].apply(
    lambda x: np.fromstring(x[1:-1], sep=" ", dtype=float)
)

# Parameter estimation

This procedure should be very similar to `3-1_behavioural-modeling_fitting.py`

In [71]:
subjects = np.unique(trials_synth["subject"])[:5]
n_subjects = len(subjects)
subjects

array(['0-eu', '0-gaze-baseline-dyn', '0-gaze-baseline-stat',
       '0-glickman1layer', '0-mdft'], dtype=object)

In [34]:
def input_generator(data, subjects, models, verbose=False):
    "Generates subject-wise inputs to fit_subject_model function."
    for model in models:
        if VERBOSE > 0:
            _ = model(data=data)
            print("{}: Parameter estimation...".format(_.label))
            del _
        for subject in subjects:
            # Subset subject data
            subject_data = data[data["subject"] == subject].copy()
            # Initialize model instance
            subject_model = model(data=subject_data)
            subject_model.subject = subject
            # Yield everything as a tuple to input into fit_subject_model function
            yield subject, subject_model

In [43]:
from functools import partial

N_CORES = 4
N_RUNS = 1
OPTMETHOD = "differential_evolution"
OUTPUT_DIR = join("..", "results", "S_recoveries")
LABEL = ""
SEED = 1
OVERWRITE = True
VERBOSE = 2

In [44]:
def fit_subject_model(
    variable_input,
    n_runs=1,
    optmethod="minimize",
    output_dir="",
    label="",
    seed=None,
    overwrite=False,
    verbose=False,
):
    """
    Fit single subject model.
    variable_input is generated by input_generator() and contains
    the subject ID, the model instance with attached data.
    """
    # Unpack input
    subject, subject_model = variable_input
    model_label = subject_model.label.replace(" ", "-").lower()

    # Organize output folders
    estimates_output_folder = join(output_dir, "estimates", model_label)
    if not exists(estimates_output_folder):
        makedirs(estimates_output_folder)

    # Create output filenames
    estimates_output_file = "estimates_{}_{}{}.csv".format(model_label, subject, label)
    estimates_output_path = join(estimates_output_folder, estimates_output_file)
    # Parameter estimation
    if not overwrite and exists(estimates_output_path):
        if verbose > 0:
            print(
                "Found existing estimates at '{}'. Skipping estimation...".format(
                    estimates_output_path
                )
            )
        estimates_df = pd.read_csv(estimates_output_path)
        estimates = [
            estimates_df[parameter].values[0]
            for parameter in subject_model.parameter_names
        ]
        estimates_df["bic"] = (
            2 * estimates_df["nll"]
            + np.log(subject_model.n_trials) * subject_model.n_parameters
        )
    else:
        estimates, nll = subject_model.fit(
            method=optmethod, n_runs=n_runs, seed=seed, verbose=verbose
        )
        bic = 2 * nll + np.log(subject_model.n_trials) * subject_model.n_parameters
        # Put fitting results into DataFrame row
        estimates_df = pd.DataFrame(
            dict(subject=subject, nll=nll, bic=bic, model=model_label), index=[subject]
        )
        for parameter, estimate in zip(subject_model.parameter_names, estimates):
            estimates_df[parameter] = estimate
        estimates_df.to_csv(estimates_output_path, index=False)

    return estimates_df

In [45]:
# Fix some arguments of the fitting function, so that it only takes subject data as argument
fit = partial(
    fit_subject_model,
    n_runs=N_RUNS,
    optmethod=OPTMETHOD,
    output_dir=OUTPUT_DIR,
    label=LABEL,
    seed=SEED,
    overwrite=OVERWRITE,
    verbose=VERBOSE,
)

In [46]:
from os import makedirs
from os.path import exists

In [47]:
N_CORES = 1

In [50]:
# Run fitting in parallel if multiple cores specified
if N_CORES > 1:
    from multiprocessing import Pool

    pool = Pool(N_CORES)
    # next line can use `map`, then outputs work fine, or `imap_unordered` which maps better, but messes with output concatenation.
    results = pool.map(fit, input_generator(trials_synth, subjects, models, VERBOSE))
    all_estimates = [result for result in results]
# otherwise sequential
else:
    all_estimates = []
    for s, (subject, subject_model) in enumerate(
        input_generator(trials_synth, subjects, models, VERBOSE)
    ):
        if VERBOSE > 0:
            print("Subject {} ({}/{})".format(subject, s + 1, n_subjects))
            subject_model.subject = 1
        estimates = fit((subject, subject_model))
        all_estimates.append(estimates)

# Collect and save results
all_estimates_output_path = join(OUTPUT_DIR, "estimates", "estimates{}.csv").format(
    LABEL
)
all_estimates = pd.concat(all_estimates, sort=True)
all_estimates.to_csv(all_estimates_output_path, index=False)

EU: Parameter estimation...
0-eu
Subject 0-eu (1/5)
EU	Subject 1	Run 1 of 1 (100%)
0-gaze-baseline-dyn
Subject 0-gaze-baseline-dyn (2/5)
EU	Subject 1	Run 1 of 1 (100%)
0-gaze-baseline-stat
Subject 0-gaze-baseline-stat (3/5)
EU	Subject 1	Run 1 of 1 (100%)
0-glickman1layer
Subject 0-glickman1layer (4/5)
EU	Subject 1	Run 1 of 1 (100%)
0-mdft
Subject 0-mdft (5/5)
EU	Subject 1	Run 1 of 1 (100%)
PT: Parameter estimation...
0-eu
Subject 0-eu (6/5)
PT	Subject 1	Run 1 of 1 (100%)
0-gaze-baseline-dyn
Subject 0-gaze-baseline-dyn (7/5)
PT	Subject 1	Run 1 of 1 (100%)
0-gaze-baseline-stat
Subject 0-gaze-baseline-stat (8/5)
PT	Subject 1	Run 1 of 1 (100%)
0-glickman1layer
Subject 0-glickman1layer (9/5)
PT	Subject 1	Run 1 of 1 (100%)
0-mdft
Subject 0-mdft (10/5)
PT	Subject 1	Run 1 of 1 (100%)
MDFT: Parameter estimation...
0-eu
Subject 0-eu (11/5)
MDFT	Subject 1	Run 1 of 1 (100%)


/Users/felixmolter/Desktop/gda-context/src/behavioural_models/mdft.py:177: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  p2, abserror = quad(Fc, 0, r, args=(m1, m2, s1, s2))
/Users/felixmolter/Desktop/gda-context/src/behavioural_models/mdft.py:59: RuntimeWarning: invalid value encountered in sqrt
  s1 = np.sqrt(Lomega[0, 0])
/Users/felixmolter/Desktop/gda-context/src/behavioural_models/mdft.py:60: RuntimeWarning: invalid value encountered in sqrt
  s2 = np.sqrt(Lomega[1, 1])
/Users/felixmolter/Desktop/gda-context/src/behavioural_models/mdft.py:177: IntegrationWarning: Extremely bad integrand behavior occurs at some points of the
  integration interval.
  p2, abserror = quad(Fc, 0, r, args=(m1, m2, s1, s2))
/Users/felixmolter/Desktop/gda-context/src/behavioural_models/m

KeyboardInterrupt: 

In [52]:
trials_synth.columns

Index(['subject', 'block', 'trial', 'effect', 'target', 'pA', 'pB', 'pC', 'mA',
       'mB', 'mC', 'dwell_A', 'dwell_B', 'dwell_C', 'dwell_p', 'dwell_m',
       'dwell_Ap', 'dwell_Am', 'dwell_Bp', 'dwell_Bm', 'dwell_Cp', 'dwell_Cm',
       'dwell_total', 'dwell_target', 'dwell_competitor', 'dwell_decoy',
       'fixated_alternatives', 'fixated_attributes', 'fixation_durations',
       'choice', 'predicted_choiceprob_A', 'predicted_choiceprob_B',
       'predicted_choiceprob_C', 'alpha_gen', 'beta_gen', 'gamma_gen', 'w_gen',
       'wd_gen', 'phi1_gen', 'phi2_gen', 'sig2_gen', 'lam_gen', 'theta_gen'],
      dtype='object')

In [54]:
trials_synth["fixated_alternatives"][0]

'[1 0 0 0 0 0 0 0 1 1 1 1 2 2 2 2 2 1 1 1 1 0 0 0 0 1 1 2 2 1]'

In [62]:
np.fromstring("1 0 0", sep=" ", dtype=int)

array([1, 0, 0])

In [63]:
np.fromstring(trials_synth["fixated_alternatives"][0][1:-1], sep=" ", dtype=int)

array([1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 2, 2, 1])

0        [1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, ...
1        [2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
2        [1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, ...
3        [2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, ...
4        [2, 0, 0, 0, 0, 0, 2, 2, 1, 1, 2, 0, 2, 0, 1, ...
                               ...                        
53929    [2, 1, 1, 1, 2, 0, 2, 2, 2, 0, 0, 1, 1, 1, 1, ...
53930    [0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, ...
53931    [0, 1, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, ...
53932    [0, 1, 0, 1, 1, 0, 0, 0, 2, 2, 0, 2, 2, 0, 1, ...
53933    [2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ...
Name: fixated_alternatives, Length: 53934, dtype: object